### NLP Challenge

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

def warn(*args, **kwargs): #remove warnings
    pass
import warnings
warnings.warn = warn

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/belalabusaleh/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
thursday = gutenberg.raw('chesterton-thursday.txt')
sense = gutenberg.raw('austen-sense.txt')

# The Chapter indicator is idiosyncratic
thursday = re.sub(r'Chapter \d+', '', thursday)
sense = re.sub(r'CHAPTER .*', '', sense)
    
thursday = text_cleaner(thursday[:int(len(thursday)/5)])
sense = text_cleaner(sense[:int(len(sense)/5)])

In [6]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
thur_doc = nlp(thursday)
sense_doc = nlp(sense)

In [7]:
# Group into sentences.
thur_sent = [[sent, "Chesterton"] for sent in thur_doc.sents]
sense_sent = [[sent, "Austen"] for sent in sense_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(thur_sent + sense_sent)
sentences.head()

,0,1
0,"(To, Edmund, Clerihew, Bentley, A, cloud, was,...",Chesterton
1,"(Science, announced, nonentity, and, art, admi...",Chesterton
2,"(Like, the, white, lock, of, Whistler, ,, that...",Chesterton
3,"(Life, was, a, fly, that, faded, ,, and, death...",Chesterton
4,"(They, twisted, even, decent, sin, to, shapes,...",Chesterton


In [8]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
thurwords = bag_of_words(thur_doc)
sensewords = bag_of_words(sense_doc)

# Combine bags to create a set of unique words.
common_words = set(thurwords + sensewords)

In [9]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350
Processing row 1400
Processing row 1450
Processing row 1500
Processing row 1550
Processing row 1600
Processing row 1650
Processing row 1700
Processing row 1750
Processing row 1800
Processing row 1850
Processing row 1900
Processing row 1950
Processing row 2000
Processing row 2050
Processing row 2100
Processing row 2150
Processing row 2200
Processing row 2250


,flicker,admittance,indelicacy,impressive,sincerely,overcome,arrive,unanswerable,knock,Christmas,...,inheritor,friendship,blast,conservative,despotism,accusation,applaud,threaten,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(To, Edmund, Clerihew, Bentley, A, cloud, was,...",Chesterton
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Science, announced, nonentity, and, art, admi...",Chesterton
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Like, the, white, lock, of, Whistler, ,, that...",Chesterton
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Life, was, a, fly, that, faded, ,, and, death...",Chesterton
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(They, twisted, even, decent, sin, to, shapes,...",Chesterton


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.25,)



In [25]:
from sklearn import ensemble
rfc = ensemble.RandomForestClassifier()
train = rfc.fit(x_train, y_train)

print('Training set score:', rfc.score(x_train, y_train))
print('Test set score:', rfc.score(x_test, y_test))

scores = cross_val_score(rfc, x_train, y_train, cv=5)
print("\nCross Val Forest Classifier: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = rfc.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training set score: 0.8885613207547169
Test set score: 0.7773851590106007

Cross Val Forest Classifier: 0.76 (+/- 0.02)

              precision    recall  f1-score   support

      Austen       0.75      0.91      0.82       321
  Chesterton       0.83      0.61      0.70       245

   micro avg       0.78      0.78      0.78       566
   macro avg       0.79      0.76      0.76       566
weighted avg       0.79      0.78      0.77       566



In [23]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(x_train, y_train)

print('Training set score:', lr.score(x_train, y_train))
print('Test set score:', lr.score(x_test, y_test))

scores = cross_val_score(lr, x_train, y_train, cv=5)
print("\nCross Val Logistic Regression: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = lr.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

(1696, 3214) (1696,)
Training set score: 0.8856132075471698
Test set score: 0.823321554770318

Cross Val Logistic Regression: 0.78 (+/- 0.02)

              precision    recall  f1-score   support

      Austen       0.79      0.94      0.86       321
  Chesterton       0.89      0.67      0.77       245

   micro avg       0.82      0.82      0.82       566
   macro avg       0.84      0.81      0.81       566
weighted avg       0.83      0.82      0.82       566



In [31]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(x_train, y_train))
print('Test set score:', clf.score(x_test, y_test))

scores = cross_val_score(clf, x_train, y_train, cv=5)
print("\nCross Val GradientBoosting: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training set score: 0.5607311320754716
Test set score: 0.5388692579505301

Cross Val GradientBoosting: 0.71 (+/- 0.03)

              precision    recall  f1-score   support

      Austen       0.56      0.85      0.68       321
  Chesterton       0.40      0.13      0.19       245

   micro avg       0.54      0.54      0.54       566
   macro avg       0.48      0.49      0.43       566
weighted avg       0.49      0.54      0.47       566



### Logistic Regression with Bag of words works best here!


### Lets try tf-idf now

In [32]:
import nltk
from nltk.corpus import gutenberg
nltk.download('punkt')
nltk.download('gutenberg')
import re
from sklearn.model_selection import train_test_split

#reading in the data, this time in the form of paragraphs
sense_1=gutenberg.paras('austen-sense.txt')
#processing
sense_paras=[]
for paragraph in sense_1:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    sense_paras.append(' '.join(para))

print(sense_paras[0:4])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/belalabusaleh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/belalabusaleh/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
['[ Sense and Sensibility by Jane Austen 1811 ]', 'CHAPTER 1', 'The family of Dashwood had long been settled in Sussex .', 'By a former marriage , Mr . Henry Dashwood had one son : by his present lady , three daughters .']


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test = train_test_split(sense_paras,
                                   test_size=0.25)

#Set our parameters
vectorizer = TfidfVectorizer(max_df=0.9, # drop words that occur in more than half the paragraphs
                             min_df=3, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
sense_paras_tfidf=vectorizer.fit_transform(sense_paras)
print("Number of features: %d" % sense_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(sense_paras_tfidf, 
                                              test_size=0.25)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 1311
Original sentence: " Nor I ," answered Marianne with energy , " our situations then are alike .
Tf_idf vector: {'lucy': 0.27039798753420413, 'wished': 0.34952461452398426, 'pressed': 0.44840296631250753, 'reason': 0.36540692802784613, 'happiness': 0.33450540357651476, 'talk': 0.3619037298085233, 'obliged': 0.34952461452398426, 'elinor': 0.32418266162976966}


### Okay, now we have our vectors, with one vector per paragraph.
It's time to do some dimension reduction. We use the Singular Value Decomposition (SVD) function from sklearn rather than PCA because we don't want to mean-center our variables (and thus lose sparsity):


In [60]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer. We are going to reduce the feature space from 1942 to 130.
svd= TruncatedSVD(250)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:3])



Percent variance captured by all components: 65.28344609780035
Component 0:
" Being very sure I have long lost your affections , I have thought myself at liberty to bestow my own on another , and have no doubt of being as happy with him as I once used to think I might be with you ; but I scorn to accept a hand while the heart was another ' s .                                                                                                                                       1.0
Mrs . Dashwood , not less watchful of what passed than her daughter , but with a mind very differently influenced , and therefore watching to very different effect , saw nothing in the Colonel ' s behaviour but what arose from the most simple and self - evident sensations , while in the actions and words of Marianne she persuaded herself to think that something more than gratitude already dawned .    1.0
" And you will never see me otherwise .                                                                     

In [61]:
#Look at testing data

# Run SVD on the testing data, then project the testing data.
X_test_lsa = lsa.fit_transform(X_test_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_test_lsa,index=X_test)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:3])

Percent variance captured by all components: 86.16843205471466
Component 0:
" The Colonel is a ninny , my dear ; because he has two thousand a - year himself , he thinks that nobody else can marry on less .    1.0
" Well , Marianne ," said Elinor , as soon as he had left them , " for ONE morning I think you have done pretty well .                1.0
" He would certainly have done more justice to simple and elegant prose .                                                             1.0
Name: 0, dtype: float64
Component 1:
" What a charming thing it is that Mrs . Dashwood can spare you both for so long a time together !"     0.460554
" Come Colonel ," said Mrs . Jennings , " before you go , do let us know what you are going about ."    0.395159
" Oh !                                                                                                  0.392633
Name: 1, dtype: float64
Component 2:
" Perhaps ," said Elinor , smiling , " we may come to the same point .                           

### Just for fun lets use word2vec for word comparisons

In [62]:
from nltk.corpus import gutenberg, stopwords

In [64]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [65]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
austen_doc = nlp(austen_clean)

In [83]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower() #convert all to lowercase
        for token in sentence
        if not token.is_stop #remove stop words
        and not token.is_punct #remove punctuation
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['lady', 'russell', 'steady', 'age', 'character', 'extremely', 'provide', 'thought', 'second', 'marriage', 'need', 'apology', 'public', 'apt', 'unreasonably', 'discontent', 'woman', 'marry', 'sir', 'walter', 'continue', 'singleness', 'require', 'explanation']
We have 9298 sentences and 900000 tokens.


In [90]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel
    min_count=5,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [91]:
# List of words in model.
vocab = model.wv.vocab.keys()

#woman is to lady as man is to ???: print result
print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('lady', 'gentleman'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('nineteen', 0.9058294296264648), ('hitherto', 0.9029128551483154), ('confident', 0.9012742042541504), ('suddenly', 0.9002410173416138), ('afternoon', 0.9000874757766724), ('jealousy', 0.8993202447891235), ('surprised', 0.8987928032875061), ('forth', 0.8986408710479736), ('suitable', 0.8968305587768555), ('intelligence', 0.8958185911178589)]
0.75374824
marriage
